In [101]:
import pandas as pd
import numpy as np
import re
import datetime
##https://www.kaggle.com/vijayuv/onlineretail?select=OnlineRetail.csv

In [102]:
ttl = pd.read_csv('/Users/APinkerton/NYC_DS_A/projects/proj_1/OnlineRetail.csv',encoding= 'unicode_escape')
ttl.columns
ttl.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [103]:
np.unique(ttl['InvoiceDate'])

array(['1/10/2011 10:04', '1/10/2011 10:07', '1/10/2011 10:08', ...,
       '9/9/2011 9:13', '9/9/2011 9:38', '9/9/2011 9:52'], dtype=object)

In [104]:
date = ttl.InvoiceDate.str.split(" ",expand=True)
full = pd.merge(ttl, date, left_index=True, right_index=True)
full.drop(columns=['InvoiceDate',1],inplace=True)
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12/1/2010
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12/1/2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12/1/2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12/1/2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12/1/2010


In [105]:
full['sls$']=full['Quantity']*full['UnitPrice']
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,sls$
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12/1/2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12/1/2010,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12/1/2010,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12/1/2010,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12/1/2010,20.34


In [165]:
full[0]=pd.to_datetime(full[0])
full['mo']=full[0].dt.month
full['day']=full[0].dt.day
full['year']=full[0].dt.year
full.columns
full.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,sls$,mo,day,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,12,1,2010
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,12,1,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010


In [182]:
dates = pd.read_csv('/Users/APinkerton/NYC_DS_A/projects/proj_1/days.csv')
dates['date']=pd.to_datetime(dates['date'])
dates.head()

,date,Wk
0,2010-12-01,DEC Wk 1
1,2010-12-02,DEC Wk 1
2,2010-12-03,DEC Wk 1
3,2010-12-04,DEC Wk 1
4,2010-12-05,DEC Wk 2


In [192]:
new = pd.merge(full,dates,left_on=0,right_on='date')
new.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,sls$,mo,day,year,date,Wk
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,12,1,2010,2010-12-01,DEC Wk 1
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,12,1,2010,2010-12-01,DEC Wk 1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1


In [212]:
new['454_mo'] = new['Wk'].astype(str).str[0:3]
new.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,0,sls$,mo,day,year,date,Wk,454_mo
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,2010-12-01,15.30,12,1,2010,2010-12-01,DEC Wk 1,DEC
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1,DEC
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,2010-12-01,22.00,12,1,2010,2010-12-01,DEC Wk 1,DEC
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1,DEC
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,2010-12-01,20.34,12,1,2010,2010-12-01,DEC Wk 1,DEC


In [216]:
new.groupby(['StockCode','Description', 'Wk']).agg('sum').head()
week_ttl = new.groupby(['Wk']).agg('sum')
cal_month_ttl = new.groupby(['mo']).agg('sum')
month_454_ttl = new.groupby(['454_mo']).agg('sum')
new['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
                                      ... 
PINK ALLIUM  ARTIFICIAL FLOWER           1
cracked                                  1
BAROQUE BUTTERFLY EARRINGS MONTANA       1
IVORY PANTRY HANGING LAMP                1
wrongly sold as sets                     1
Name: Description, Length: 4223, dtype: int64

In [113]:
full.head

<bound method NDFrame.head of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

        UnitPrice  CustomerID         Country